# Parameter frame

The module provides an interface for managing solution parameters.
It allows for the structured storage and retrieval of parameter sets from a database.


In [1]:
import sys
sys.path.append('../')
from draft_modules.parameterframe import FileTypeHandler, ParameterFrame, DatabaseConnector
import pandas as pd

## Content

1. Processing and reconstructing yaml and txt with FileTypeHandler
2. Assembling parameter sets and adding to solution with ParameterFrame
3. Commiting and pushing solutions with DatabaseConnector

## Example data

In [2]:
yaml_content = """
key_1: value_1
key_2:
  subkey_1: value_1
  subkey_2: value_2
key_3:
  - val_1
  - val_2
"""

txt_content = """
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Aliquam ac est a dolor porta mollis.
Integer quis egestas augue. Donec fermentum magna in neque venenatis, ut pulvinar lorem porta.
Etiam dictum tristique ligula vel convallis. Donec pretium porta vulputate. Vestibulum vel justo
fringilla, eleifend arcu aliquet, hendrerit urna. Sed sed vehicula quam. Donec interdum at magna eget
gravida. Vestibulum eleifend gravida semper. Nullam arcu lectus, tempus ut posuere ut, euismod ac nulla.
Donec condimentum diam lectus. Nullam tincidunt libero id metus placerat suscipit. In vel ornare arcu.

Integer malesuada posuere tempor. Nullam pharetra mollis justo et tempus. Maecenas molestie,
orci a rhoncus placerat, enim tellus ornare ligula, quis facilisis lectus mi in est.
Nulla auctor purus dolor, quis hendrerit ligula tincidunt eu. Suspendisse elit lacus, suscipit
ullamcorper lacus et, tristique accumsan sem. Quisque a metus sapien. Cras mattis lorem sed tortor
finibus euismod. Sed non lacus pretium, condimentum urna et, facilisis dui.
"""

param_path_1 = "param_1.yaml"
param_path_2 = "param_2.yaml"

with open(param_path_1, 'w', encoding='utf-8') as file:
  file.write(yaml_content)

with open(param_path_2, 'w', encoding='utf-8') as file:
  file.write(txt_content)

params_path = "../tests/parameterframe/example_configs"

### 1. Processing and reconstructing yaml and txt with FileTypeHandler

In [3]:
pf = ParameterFrame(
    params_path = params_path
)

pf.process_parameters_from_files()

### 2. Assembling parameter sets and adding to solution with ParameterFrame

In [5]:
pf.make_parameter_set(
    parameter_set_name="test_set",
    parameter_set_description="example parameters for test purposes",
    parameter_names=['param_1','param_2','param_10']
)

In [6]:
pf.param_sets

{'test_set': {'parameter_set': [{'parameter_set_id': 'e73396d2601456e8bb84a811aaec5fb46dda83d70f9cd706ec58d5ea76e0d5dd',
    'parameter_id': '4cea5b09e77da310c5105978f2ceea5c5d8c9c7b65d0e00b45135ea90fc011af'},
   {'parameter_set_id': 'e73396d2601456e8bb84a811aaec5fb46dda83d70f9cd706ec58d5ea76e0d5dd',
    'parameter_id': 'bf11768decb1d0204e2636edd05c354573d473e67f1b048369b2ee99c865bf5f'},
   {'parameter_set_id': 'e73396d2601456e8bb84a811aaec5fb46dda83d70f9cd706ec58d5ea76e0d5dd',
    'parameter_id': '9a4a3ace265c9bf2facc0044ca24260c42805c6e7b2a608dfd2f56a54d9d36be'}],
  'parameter_set_description': [{'parameter_set_id': 'e73396d2601456e8bb84a811aaec5fb46dda83d70f9cd706ec58d5ea76e0d5dd',
    'parameter_set_name': 'test_set',
    'parameter_set_description': 'example parameters for test purposes'}]}}

In [7]:
pf.add_solution_description(solution_name = "example_solution",
                            solution_description = "example solution for test")

In [8]:
pf.solutions['example_solution']

{'solution_id': 'ffb2fa4d1f14786e7a11641a870c3db55f08f375fb7ac00c9a2127f7cd801a60',
 'solution_description': {'solution_id': 'ffb2fa4d1f14786e7a11641a870c3db55f08f375fb7ac00c9a2127f7cd801a60',
  'solution_name': 'example_solution',
  'solution_description': 'example solution for test',
  'deployment_date': None,
  'deprication_date': None,
  'maintainers': None}}

In [9]:
pf.add_parameter_set_to_solution(solution_name="example_solution",
                                 parameter_set_name="test_set")

In [10]:
pd.DataFrame([pf.solutions['example_solution']['solution_parameter_set']['test_set']])

,solution_id,parameter_set_id,deployment_status,insertion_datetime
0,ffb2fa4d1f14786e7a11641a870c3db55f08f375fb7ac0...,e73396d2601456e8bb84a811aaec5fb46dda83d70f9cd7...,STAGING,2024-04-05 06:31:55


### 3. Commiting and pushing solutions with DatabaseConnector

In [11]:
pf.commit_solution(solution_name="example_solution",
                    parameter_set_names=["test_set"])

#### solution_description

In [12]:
pd.DataFrame(pf.commited_tables['ffb2fa4d1f14786e7a11641a870c3db55f08f375fb7ac00c9a2127f7cd801a60']['solution_description'])

,solution_id,solution_name,solution_description,deployment_date,deprication_date,maintainers
0,ffb2fa4d1f14786e7a11641a870c3db55f08f375fb7ac0...,example_solution,example solution for test,None,None,None


#### solution_parameter_set

In [13]:
pd.DataFrame(pf.commited_tables['ffb2fa4d1f14786e7a11641a870c3db55f08f375fb7ac00c9a2127f7cd801a60']['solution_parameter_set']['e73396d2601456e8bb84a811aaec5fb46dda83d70f9cd706ec58d5ea76e0d5dd'])

,solution_id,parameter_set_id,deployment_status,insertion_datetime
0,ffb2fa4d1f14786e7a11641a870c3db55f08f375fb7ac0...,e73396d2601456e8bb84a811aaec5fb46dda83d70f9cd7...,STAGING,2024-04-05 06:31:55


#### parameter_set

In [14]:
pd.DataFrame(pf.commited_tables['ffb2fa4d1f14786e7a11641a870c3db55f08f375fb7ac00c9a2127f7cd801a60']\
    ['parameter_set']\
        ['e73396d2601456e8bb84a811aaec5fb46dda83d70f9cd706ec58d5ea76e0d5dd'])

,parameter_set_id,parameter_id
0,e73396d2601456e8bb84a811aaec5fb46dda83d70f9cd7...,4cea5b09e77da310c5105978f2ceea5c5d8c9c7b65d0e0...
1,e73396d2601456e8bb84a811aaec5fb46dda83d70f9cd7...,bf11768decb1d0204e2636edd05c354573d473e67f1b04...
2,e73396d2601456e8bb84a811aaec5fb46dda83d70f9cd7...,9a4a3ace265c9bf2facc0044ca24260c42805c6e7b2a60...


#### parameter_set_description

In [15]:
pd.DataFrame(pf.commited_tables['ffb2fa4d1f14786e7a11641a870c3db55f08f375fb7ac00c9a2127f7cd801a60']\
    ['parameter_set_description']\
        ['e73396d2601456e8bb84a811aaec5fb46dda83d70f9cd706ec58d5ea76e0d5dd'])

,parameter_set_id,parameter_set_name,parameter_set_description
0,e73396d2601456e8bb84a811aaec5fb46dda83d70f9cd7...,test_set,example parameters for test purposes


#### parameter_description

In [16]:
pf.commited_tables['ffb2fa4d1f14786e7a11641a870c3db55f08f375fb7ac00c9a2127f7cd801a60']['parameter_description']['e73396d2601456e8bb84a811aaec5fb46dda83d70f9cd706ec58d5ea76e0d5dd']

{'4cea5b09e77da310c5105978f2ceea5c5d8c9c7b65d0e00b45135ea90fc011af': [{'parameter_id': '4cea5b09e77da310c5105978f2ceea5c5d8c9c7b65d0e00b45135ea90fc011af',
   'parameter_name': 'param_1',
   'parameter_description': '',
   'file_name': 'param_1.yaml',
   'file_type': 'yaml'}],
 'bf11768decb1d0204e2636edd05c354573d473e67f1b048369b2ee99c865bf5f': [{'parameter_id': 'bf11768decb1d0204e2636edd05c354573d473e67f1b048369b2ee99c865bf5f',
   'parameter_name': 'param_2',
   'parameter_description': '',
   'file_name': 'param_2.yaml',
   'file_type': 'yaml'}],
 '9a4a3ace265c9bf2facc0044ca24260c42805c6e7b2a608dfd2f56a54d9d36be': [{'parameter_id': '9a4a3ace265c9bf2facc0044ca24260c42805c6e7b2a608dfd2f56a54d9d36be',
   'parameter_name': 'param_10',
   'parameter_description': '',
   'file_name': 'param_10.txt',
   'file_type': 'txt'}]}

#### parameter_description

In [17]:
pd.DataFrame([tab for param_id, tab_list in pf.commited_tables['ffb2fa4d1f14786e7a11641a870c3db55f08f375fb7ac00c9a2127f7cd801a60']\
    ['parameter_description']\
        ['e73396d2601456e8bb84a811aaec5fb46dda83d70f9cd706ec58d5ea76e0d5dd'].items()\
            for tab in tab_list])

,parameter_id,parameter_name,parameter_description,file_name,file_type
0,4cea5b09e77da310c5105978f2ceea5c5d8c9c7b65d0e0...,param_1,,param_1.yaml,yaml
1,bf11768decb1d0204e2636edd05c354573d473e67f1b04...,param_2,,param_2.yaml,yaml
2,9a4a3ace265c9bf2facc0044ca24260c42805c6e7b2a60...,param_10,,param_10.txt,txt


#### parameter_attribute

In [18]:
pd.DataFrame([tab for param_id, tab_list in pf.commited_tables['ffb2fa4d1f14786e7a11641a870c3db55f08f375fb7ac00c9a2127f7cd801a60']\
    ['parameter_attribute']\
        ['e73396d2601456e8bb84a811aaec5fb46dda83d70f9cd706ec58d5ea76e0d5dd'].items() \
            for tab in tab_list])

,parameter_id,attribute_id,previous_attribute_id
0,4cea5b09e77da310c5105978f2ceea5c5d8c9c7b65d0e0...,ee25af17445d7622cbf61a5b9424246a1f3104704b68bd...,None
1,4cea5b09e77da310c5105978f2ceea5c5d8c9c7b65d0e0...,8b5b2be24e60ba407b90967820da8a1385a6d67691a02b...,None
2,4cea5b09e77da310c5105978f2ceea5c5d8c9c7b65d0e0...,52ea872c99c586530348ba8902dcab831761673d25cf1c...,None
3,bf11768decb1d0204e2636edd05c354573d473e67f1b04...,7d5ee0e0cd00c3703e5f346c6887baf503faaf9fe09077...,None
4,bf11768decb1d0204e2636edd05c354573d473e67f1b04...,ee25af17445d7622cbf61a5b9424246a1f3104704b68bd...,7d5ee0e0cd00c3703e5f346c6887baf503faaf9fe09077...
5,bf11768decb1d0204e2636edd05c354573d473e67f1b04...,3367512147bf19ae99c986b356af11dcdc067376aa1b79...,7d5ee0e0cd00c3703e5f346c6887baf503faaf9fe09077...
6,bf11768decb1d0204e2636edd05c354573d473e67f1b04...,341769820d8937a5c9f9b980eefca37f3f37fcc6fd01c6...,7d5ee0e0cd00c3703e5f346c6887baf503faaf9fe09077...
7,bf11768decb1d0204e2636edd05c354573d473e67f1b04...,2e8b00e571f9d835d3f022a9ff49b9779034ab21bffdcd...,341769820d8937a5c9f9b980eefca37f3f37fcc6fd01c6...
8,bf11768decb1d0204e2636edd05c354573d473e67f1b04...,ecd93cf051988b23b3590415f4e7d550de264600d7d2af...,341769820d8937a5c9f9b980eefca37f3f37fcc6fd01c6...
9,9a4a3ace265c9bf2facc0044ca24260c42805c6e7b2a60...,9a4a3ace265c9bf2facc0044ca24260c42805c6e7b2a60...,None


#### attribute_values

In [19]:
pd.DataFrame([tab for param_id, tab_list in pf.commited_tables['ffb2fa4d1f14786e7a11641a870c3db55f08f375fb7ac00c9a2127f7cd801a60']\
    ['attribute_values']\
        ['e73396d2601456e8bb84a811aaec5fb46dda83d70f9cd706ec58d5ea76e0d5dd'].items() \
            for tab in tab_list])

,attribute_id,attribute_name,attribute_value,attribute_value_type
0,ee25af17445d7622cbf61a5b9424246a1f3104704b68bd...,name,Some name,str
1,8b5b2be24e60ba407b90967820da8a1385a6d67691a02b...,age,111,int
2,52ea872c99c586530348ba8902dcab831761673d25cf1c...,country,Some land,str
3,7d5ee0e0cd00c3703e5f346c6887baf503faaf9fe09077...,employee,"{'name': 'Some name', 'id': 10293, 'contact': ...",dict
4,ee25af17445d7622cbf61a5b9424246a1f3104704b68bd...,name,Some name,str
5,3367512147bf19ae99c986b356af11dcdc067376aa1b79...,id,10293,int
6,341769820d8937a5c9f9b980eefca37f3f37fcc6fd01c6...,contact,"{'email': 'some.name@example.com', 'phone': '+...",dict
7,2e8b00e571f9d835d3f022a9ff49b9779034ab21bffdcd...,email,some.name@example.com,str
8,ecd93cf051988b23b3590415f4e7d550de264600d7d2af...,phone,+1234567890,str
9,9a4a3ace265c9bf2facc0044ca24260c42805c6e7b2a60...,None,"\X/Fc7;/v`6joU5z*n{35zFB<<6BMC,}/_04],>v$Jr2&0...",str


In [20]:
pf.database_connector.add_entries(table_name = 'attribute_values',
             entries = pf.commited_tables['ffb2fa4d1f14786e7a11641a870c3db55f08f375fb7ac00c9a2127f7cd801a60']['attribute_values']['e73396d2601456e8bb84a811aaec5fb46dda83d70f9cd706ec58d5ea76e0d5dd']['4cea5b09e77da310c5105978f2ceea5c5d8c9c7b65d0e00b45135ea90fc011af'])

True

In [22]:
pf.database_connector.get_entries(table_name = 'attribute_values',
               return_keys = ['attribute_id',
                                'attribute_name',
                                'attribute_value',
                                'attribute_value_type'])

[{'attribute_id': 'ee25af17445d7622cbf61a5b9424246a1f3104704b68bd31b9b7532471d492e5',
  'attribute_name': 'name',
  'attribute_value': 'Some name',
  'attribute_value_type': 'str'},
 {'attribute_id': '8b5b2be24e60ba407b90967820da8a1385a6d67691a02bc663703160ef655101',
  'attribute_name': 'age',
  'attribute_value': '111',
  'attribute_value_type': 'int'},
 {'attribute_id': '52ea872c99c586530348ba8902dcab831761673d25cf1cb0023576820289ce6b',
  'attribute_name': 'country',
  'attribute_value': 'Some land',
  'attribute_value_type': 'str'}]

In [25]:
pf.database_connector.commit()